#  思路
1. 构造一个class Exp，生成基本的四则运算表达式Exp: 输入参数必选：两个操作数（num1, num2），一个操作符(op)构成; 可选：是否带括号（bracelet）;
2. op若不指定，默认选取范围在四个操作符中['+', '-', '*', '/',] 随机选取。
3. bracelet = True 或者 False，决定生成的Exp是否带括号。
4. 默认num1, num2的生成方法：num1,num2 若未指定，默认选取范围【0，100】，随机生成，可以是整数，或者小数；num2要求总是大于等于num1；
5. 根据 op 的不同，预留重载四种num1,num2的生成方法
6. 一个子函数生成Exp： f"({num1} {op} {num2})" if bracelet else f"{num1} {op} {num2}"
7. 一个子函数调用Exp，使用eval()方法得到计算结果

# 数的规则 

## 从num1和num2生成result
### 加法限制
1. 如果不允许负值出现，那么 num1 >= 0， num2 >= 0
2. result = num1 + num2
### 减法限制
1. 如果不允许负值出现，那么 num1 >= num2, num2 >=0
2. result = num1 - num2
### 乘法限制
1. 如果不允许负值出现，那么 那么 num1 >= 0， num2 >= 0
2. result = num1 * num2
### 除法限制
0. num2 != 0
1. 如果不允许小数出现，那么num1 = num2*R, R!=0的整数
3. 如果不允许负值出现，那么 那么 num1 >= 0， num2 > 0
4. result = num1 / num2
#### 余数除法
4. 只能是整数
5. 不允许负值出现，那么 那么 num1 >= 0， num2 > 0
6. result (div...remain) = num1 / num2

## 从result和num2生成num1 -- 只适用于加法和减法 


## 特殊配置

In [16]:
# MOD 允许除法余数形式
# MULTI_ZERO 出现多个0的乘除法
# MULTI_25 和25相关的乘除法
# QUICK 巧算
# None: no special

SPECIAL = 'QUICK' # 'MOD', 'MULTI_ZERO','MULTI_25'  ‘QUICK’

# 难度等级
DIFF = 2

# debug show
debug = True

# 只加减 或者 只乘除
OP_SELECT = 'AS_ONLY' # 'MD_ONLY' 'ALL' 'AS_ONLY' 
OP_BIAS   = 50 # [0,100]， 0： ‘+‘ only or '*' only; 100 '-' only or '/' only

if OP_SELECT == 'AS_ONLY' and  SPECIAL in ['MOD', 'MULTI_ZERO','MULTI_25']:
    assert False, f"not matched OP_SELECT = {OP_SELECT}, SPECIAL = {SPECIAL}"


In [2]:
# 加减法的极值

if   DIFF == 1: # 三年级上
    AS_MAX = 30
    
    MD_MIN_LOW  = 2
    MD_MIN_HIGH = 5
    
    MD_MAX_LOW  = 3
    MD_MAX_HIGH = 10
    
elif DIFF == 2:  # 三年级下
    AS_MAX = 60
    
    MD_MIN_LOW  = 6
    MD_MIN_HIGH = 10
    
    MD_MAX_LOW  = 9
    MD_MAX_HIGH = 20

elif DIFF == 3:  # 四年级上
    AS_MAX = 200
        
    MD_MIN_LOW  = 8
    MD_MIN_HIGH = 10
    
    MD_MAX_LOW  = 9
    MD_MAX_HIGH = 30

elif DIFF == 4:  # 四年级下
    AS_MAX = 300
        
    MD_MIN_LOW  = 6
    MD_MIN_HIGH = 10
    
    MD_MAX_LOW  = 9
    MD_MAX_HIGH = 299

elif DIFF == 11:  # 允许负数 todo
    pass
else:
    pass

In [3]:
import random

TYPE_DICT = ['+', '-', '*', '/']
AS_DICT = ['+', '-']
MD_DICT = ['*', '/']

# 占位符, pdf友好
NULL_2 = '\u0000' * 2
NULL_4 = '\u0000' * 4
NULL_8 = '\u0000' * 8
NULL_12 = '\u0000' * 12

# 占位符, print友好
# NULL_4 = ' ' * 4
# NULL_8 = ' ' * 8

# 几个特殊常数
ZERO = 0
ONE  = 1
TWO  = 2
THREE = 3
FOUR = 4
FIVE = 5
NINE = 9
TEN  = 10
FIVETEEN = 15
HUND = 100


In [4]:
def biased_random(threshold:int):
    '''干预随机选择结果, 基于阈值决定多少 50为公平随机'''
    assert threshold >= 0 and threshold <= 100
    # 将阈值范围0-99映射到0.0-1.0的概率范围
    probability_of_one = (threshold / 100.0)
    # 生成一个0到1之间的随机数
    random_number = random.random()
    # 如果随机数小于等于阈值对应的1的概率，则输出1，否则输出0
    if random_number <= probability_of_one:
        return 1
    else:
        return 0

In [5]:
def op_choice(choice: str, bias:int):
    if   choice == 'AS_ONLY':
        return AS_DICT[biased_random(bias)]
    elif choice == 'MD_ONLY':
        return MD_DICT[biased_random(bias)]
    else: 
        return random.choice(AS_DICT+MD_DICT)


In [6]:
def generate_number(min_val=ZERO, max_val=HUND, integer=True, avoidZero=False):
    '''随机生成一个介于 (min_val, max_val) 之间的整数或者小数'''
    while True:
        rnd = random.randint(min_val, max_val) if integer else random.uniform(min_val, max_val)
        if not (avoidZero and rnd == 0):
            break
            
    return rnd

In [7]:
class Exp:
    def __init__(self, result=100, num1=None, num2=None, op=None, bracelet:bool=False, integer:bool=True, positive:bool=True):
        self.op = op if op is not None else op_choice(OP_SELECT, OP_BIAS)
        if SPECIAL == 'MOD':
            self.integer  = True
            self.bracelet = False
            self.positive = True
            self.op = '/' # force to be division for 'MOD'

        elif SPECIAL  in ['MULTI_ZERO',  'MULTI_25'] :
            self.integer  = True
            self.bracelet = False
            self.positive = positive
        
        elif SPECIAL  == 'QUICK':
            self.integer  = True
            self.bracelet = bracelet
            self.positive = True
            
        else:
            self.integer  = integer
            self.bracelet = bracelet
            self.positive = positive
        
        self.op_str = None
        self.result = result
        self.num1 = num1
        self.num2 = num2
        self.exp = None
        self.genExp = False # a flag to make sure only gen once time

    def generate_addition_numbers_from_result(self):
        num2 = generate_number(min_val=ZERO, max_val=self.result, integer=self.integer)
        num1 = self.result - num2
        return num1, num2

    def generate_subtraction_numbers_from_result(self):
        num2 = generate_number(min_val=ZERO, max_val=AS_MAX, integer=self.integer)
        num1 = self.result + num2
        return num1, num2

    def generate_addition_numbers(self, positive):
        if positive:
            num2 = generate_number(min_val=ONE, max_val=AS_MAX, integer=self.integer)
            num1 = num2+generate_number(min_val=ONE, max_val=AS_MAX, integer=self.integer)
        else:
            num2 = generate_number(min_val=-1*AS_MAX, max_val=AS_MAX, integer=self.integer)
            num1 = num2 + generate_number(min_val=-1*AS_MAX, max_val=AS_MAX, integer=self.integer)

        return num1, num2

    def generate_subtraction_numbers(self, positive):
        if positive:
            num2 = generate_number(min_val=ZERO, max_val=AS_MAX, integer=self.integer)
            num1 = num2 + generate_number(min_val=ZERO, max_val=AS_MAX, integer=self.integer)
        else:
            num2 = generate_number(min_val=-1*AS_MAX, max_val=AS_MAX, integer=self.integer)
            num1 = num2 + generate_number(min_val=-1*AS_MAX, max_val=AS_MAX, integer=self.integer)     
        return num1, num2

    def generate_multiplication_numbers(self, positive):
        posFact = 1 if positive else -1
        num1 = generate_number(min_val=posFact*MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=self.integer)
        num2 = generate_number(min_val=posFact*MD_MIN_LOW, max_val=MD_MIN_HIGH, integer=self.integer)

        if SPECIAL  == 'MULTI_ZERO':
            num1 = num1 * random.choice([FIVE, TEN, FIVETEEN])
            num2 = num2 * 2

        if SPECIAL  == 'MULTI_25':
            num1 = num1 * random.choice([25, 50, 75])
        
        return num1, num2


    def generate_division_numbers(self, positive):
        posFact = 1 if positive else -1
        num2 = generate_number(min_val=posFact*MD_MIN_LOW, max_val=MD_MIN_HIGH, integer=self.integer, avoidZero=True)
        if self.integer:
            num1 = num2*generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=True)
        else:
            num1 = generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=False)   

        if SPECIAL  == 'MULTI_ZERO':
            num1 = num1 * random.choice([FIVE, TEN, FIVETEEN])

        if SPECIAL  == 'MULTI_25':
            num2 = random.choice([25, 50, 75])
            num1 = num2 * generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=True)
        
        # print(f"[deg div]: num1={num1}, num2={num2}")
        return num1, num2

    def generate_division_numbers_for_remain(self): # 总是正整数数
        assert self.integer == True, f"something wrong, for MOD division, must be integer "
        num2 = generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=self.integer, avoidZero=True)
        num1 = num2*generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=self.integer) \
               + generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH-1, integer=self.integer, avoidZero=True)
        return num1, num2
    
    def _genOpStr(self):
            if   self.op == '+':   self.op_str = "\uFF0B"
            elif self.op == '-':   self.op_str = "\uFF0D"
            elif self.op == '*':   self.op_str = "\u00D7"
            elif self.op == '/':   self.op_str = "\u00F7"

    def _genNums(self):
            if   self.op == '+':
                if self.result == None:
                    self.num1, self.num2 = self.generate_addition_numbers(self.positive )
                else:
                    self.num1, self.num2 = self.generate_addition_numbers_from_result()
            elif self.op == '-':
                if self.result == None:
                    self.num1, self.num2 = self.generate_subtraction_numbers(self.positive )
                else:
                    self.num1, self.num2 = self.generate_subtraction_numbers_from_result()
            elif self.op == '*':
                self.num1, self.num2 = self.generate_multiplication_numbers(self.positive )
            elif self.op == '/':
                if SPECIAL == 'MOD': # 除法余数计算表示
                    self.num1, self.num2 = self.generate_division_numbers_for_remain()
                else:
                    self.num1, self.num2 = self.generate_division_numbers(self.positive )
        
    def generate_exp(self):
        if not self.genExp: # make sure only run once time
            self._genOpStr()
            self._genNums()
            
            exp = f"{self.num1}{self.op}{self.num2}"
            self.exp     = f"({exp})" if self.bracelet else f"{exp}"  # 是否加括号
            
            exp_str = f"{self.num1:<8}{self.op_str:<4}{self.num2:>8}"
            self.exp_str = f"({exp_str})" if self.bracelet else f"{exp_str}"  # 是否加括号
            
            self.genExp     = True
        else:
            print("warning! genExp already done!")
            pass
            
        return self.exp_str
    
    def evaluate_exp(self):
        return eval(self.exp)

    def generateExpStr(self, ind:int):
        exp_str = self.generate_exp()
        
        if SPECIAL == 'MOD': # 仅供余数除法使用
            div_res = self.num1//self.num2
            div_rem = self.num1 % self.num2
            formatted_result = f"{div_res}" if div_rem == 0 else f"{div_res}...{div_rem}" 
        else:
            result  = self.evaluate_exp()
            if isinstance(result, int):
                formatted_result = result
            else:    # 如果 result 是 float 且为整数，转换为 int
                formatted_result = int(result) if result.is_integer() else f"{result:.4f}"

        formatStr = f"({ind:<2}) | {exp_str}={NULL_8}| 答案：{formatted_result}"
        # if debug: print(formatStr)
        return formatStr


In [8]:
def gen_unit_problem_str(nums:int, bracelet:bool= False, integer:bool=True, positive:bool=True):
    all_out = ''
    for i in range (nums):
        #exp = Exp(bracelet=random.randint(0,1), integer=1)
        exp = Exp(bracelet=bracelet, integer=integer, positive=positive)
        # print(f"i={i}: Generated Expression: {exp.generate_exp()}")
        # print(f"i={i}: Evaluation Result   : {exp.evaluate_exp()}")
        all_out += exp.generateExpStr(i+1) + '\n'

    if debug: print(all_out)
    return all_out

In [13]:
def gen_unit_problem_per_day(nums:int):
    out=''
    out+=f"Level:{DIFF}-{SPECIAL}:日期____， 日期____， 总题数____, 错误数____\n"
    # out+="---"*21 + "\n"
    out+=gen_unit_problem_str(nums, positive=True)
    out+="\n"*2
    return out

In [10]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

pdf_name = "multi_page_problem"+"_level"+str(DIFF)+str(SPECIAL)+".pdf"

# have to use SimSun, otherwise can not generate pdf correctly
font_name = 'SimSun'
font_path = './SimSun.ttf'

def print_problem_pdf(days, nums):
    pdfmetrics.registerFont(TTFont(font_name, font_path))  #注册字体
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(fontName=font_name, name='Sun', leading=15, fontSize=12, alignment=0))  #leading=spacing # 0=左对齐, 1=居中, 2=右对齐
      
    doc = SimpleDocTemplate(pdf_name, pagesize=letter)
    
    styles.add(ParagraphStyle(name='Address', fontName='Arial'))
    flowables = []

    for d in range(days):
        # 将换行符替换为LineBreak对象
        problem_text = gen_unit_problem_per_day(nums)
        lines = problem_text.split('\n')
        for line in lines:
            # 调整字符间距，确保对齐效果--not work
            # formatted_line = line.replace('＋', ' ＋ ').replace('－', ' － ').replace('×', ' × ').replace('÷', ' ÷ ')
            # flowables.append(Paragraph(formatted_line, styles['Sun']))
            flowables.append(Paragraph(line, styles['Sun']))

        # 添加额外的间距
        flowables.append(Spacer(1, 12))

        if d < days - 1:  # 除了最后一页外，其他每页末尾添加分页符
            flowables.append(PageBreak())

    doc.build(flowables)
    print(f"生成{pdf_name}文件")

# 使用函数生成多页PDF
days = 3  # 
nums = 40  # 
print_problem_pdf(days, nums)

(1 ) | 180     ÷         10=        | 答案：18
(2 ) | 284     ÷         14=        | 答案：20...4
(3 ) | 207     ÷         19=        | 答案：10...17
(4 ) | 190     ÷         20=        | 答案：9...10
(5 ) | 149     ÷         13=        | 答案：11...6
(6 ) | 218     ÷         20=        | 答案：10...18
(7 ) | 133     ÷         12=        | 答案：11...1
(8 ) | 221     ÷         13=        | 答案：17
(9 ) | 192     ÷         15=        | 答案：12...12
(10) | 177     ÷         12=        | 答案：14...9
(11) | 195     ÷         16=        | 答案：12...3
(12) | 236     ÷         15=        | 答案：15...11
(13) | 182     ÷         12=        | 答案：15...2
(14) | 174     ÷         18=        | 答案：9...12
(15) | 107     ÷          9=        | 答案：11...8
(16) | 193     ÷         10=        | 答案：19...3
(17) | 318     ÷         20=        | 答案：15...18
(18) | 180     ÷         18=        | 答案：10
(19) | 354     ÷         18=        | 答案：19...12
(20) | 310     ÷         15=        | 答案：20...10
(21) | 224     ÷         13=        | 答案：17..

In [19]:
def gen_combine_problem_str(nums:int, bracelet:bool= False, integer:bool=True, positive:bool=True):
    '''生成巧算'''
    all_out = ''
    for i in range (nums):
        result = AS_MAX
        exp = Exp(result=result, bracelet=bracelet, integer=integer, positive=positive)
        # print(f"i={i}: Generated Expression: {exp.generate_exp()}")
        # print(f"i={i}: Evaluation Result   : {exp.evaluate_exp()}")
        all_out += exp.generateExpStr(i+1) + '\n'

    if debug: print(all_out)
    return all_out

In [20]:
gen_combine_problem_str(nums=10, bracelet=True, integer=True, positive=True)

(1 ) | (15      ＋         45)=        | 答案：60
(2 ) | (65      －          5)=        | 答案：60
(3 ) | (73      －         13)=        | 答案：60
(4 ) | (60      －          0)=        | 答案：60
(5 ) | (72      －         12)=        | 答案：60
(6 ) | (25      ＋         35)=        | 答案：60
(7 ) | (92      －         32)=        | 答案：60
(8 ) | (104     －         44)=        | 答案：60
(9 ) | (91      －         31)=        | 答案：60
(10) | (79      －         19)=        | 答案：60



'(1 ) | (15      ＋         45)=\x00\x00\x00\x00\x00\x00\x00\x00| 答案：60\n(2 ) | (65      －          5)=\x00\x00\x00\x00\x00\x00\x00\x00| 答案：60\n(3 ) | (73      －         13)=\x00\x00\x00\x00\x00\x00\x00\x00| 答案：60\n(4 ) | (60      －          0)=\x00\x00\x00\x00\x00\x00\x00\x00| 答案：60\n(5 ) | (72      －         12)=\x00\x00\x00\x00\x00\x00\x00\x00| 答案：60\n(6 ) | (25      ＋         35)=\x00\x00\x00\x00\x00\x00\x00\x00| 答案：60\n(7 ) | (92      －         32)=\x00\x00\x00\x00\x00\x00\x00\x00| 答案：60\n(8 ) | (104     －         44)=\x00\x00\x00\x00\x00\x00\x00\x00| 答案：60\n(9 ) | (91      －         31)=\x00\x00\x00\x00\x00\x00\x00\x00| 答案：60\n(10) | (79      －         19)=\x00\x00\x00\x00\x00\x00\x00\x00| 答案：60\n'